# Methods comparison
- SVRG (with/without outer loop)
- Katyusha (with/without outer loop)

In [14]:
import operator
import time
from itertools import accumulate
from typing import Callable, Optional

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from cycler import cycler
from tqdm import tqdm

In [2]:
MANUAL_SEED = 42
np.random.seed(MANUAL_SEED)

## Data loading & preprocessing

Dataset reference can be found in README.md

In [3]:
heart_df = pd.read_csv("../data/raw/heart_attack_dataset.csv")
print(f"{len(heart_df)=}")
heart_df.head()

len(heart_df)=303


,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
def preprocess_dataset(df: pd.DataFrame) -> pd.DataFrame:
    new_df = df.copy()

    # Make labels be 1 or -1
    new_df["output"] = new_df["output"] * 2 - 1

    # Normalize columns
    new_df["age"] = new_df["age"] / new_df["age"].max()
    new_df["trtbps"] = new_df["trtbps"] / new_df["trtbps"].max()
    new_df["chol"] = new_df["chol"] / new_df["chol"].max()
    new_df["thalachh"] = new_df["thalachh"] / new_df["thalachh"].max()
    new_df["oldpeak"] = new_df["oldpeak"] / new_df["oldpeak"].max()

    return new_df


preprocessed_df = preprocess_dataset(heart_df)
preprocessed_df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,0.818182,1,3,0.725,0.413121,1,0,0.742574,0,0.370968,0,0,1,1
1,0.480519,1,2,0.650,0.443262,0,1,0.925743,0,0.564516,0,0,2,1
2,0.532468,0,1,0.650,0.361702,0,0,0.851485,0,0.225806,2,0,2,1
3,0.727273,1,1,0.600,0.418440,0,1,0.881188,0,0.129032,2,0,2,1
4,0.740260,0,0,0.600,0.627660,0,1,0.806931,1,0.096774,2,0,2,1


In [5]:
values = heart_df.drop(columns=["output"]).to_numpy()
targets = heart_df["output"].to_numpy()
print(f"{values.shape=}")
print(f"{targets.shape=}")

values.shape=(303, 13)
targets.shape=(303,)


Train/Test split

In [6]:
test_ratio = 0.1

test_length = int(test_ratio * len(targets))
train_length = len(targets) - test_length

indices = np.random.permutation(targets.shape[0])
train_idx, test_idx = indices[:train_length], indices[train_length:]

train_values, test_values = values[train_idx, :], values[test_idx, :]
train_targets, test_targets = targets[train_idx], targets[test_idx]

In [7]:
print(f"{len(train_values)=}")
print(f"{len(train_targets)=}")
print(f"{len(test_values)=}")
print(f"{len(test_targets)=}")

len(train_values)=273
len(train_targets)=273
len(test_values)=30
len(test_targets)=30


## Problems

In [18]:
class Problem:
    def get_uniformly_sampled_indices(self, n: int = 1) -> list[int]:
        raise NotImplementedError

    def get_value(self, *args, **kwargs):
        raise NotImplementedError

    def get_gradient(self, *args, **kwargs):
        raise NotImplementedError

### Binary Logistic Regression

#### Definition

Binary Logistic Regression (BLR) problem can be defined as follows:
$$\begin{equation}
\min_{w \in \mathbb{R}^d} \frac{1}{n} \sum\limits_{i=1}^n \ell (g(w, x_i), y_i) + \frac{\lambda}{2} \| w \|^2_2,
\end{equation}
$$
where $\ell(z,y) = \ln (1 + e^{-yz})$ is the loss function, $g(w, x) = w^T x$ is the model, $w$ is the model parameters, $\{x_i, y_i\}_{i=1}^n$ is the data sample from feature vectors $x_i$ and labels $y_i$, $\lambda > 0$ is the regularization parameter.

**Important Assumption**: $y$ must take values $-1$ or $+1$.

### Insights

**Lemma 1**
Let $x \in \mathbb{R}^d$. Then $X=xx^T \succeq 0$.

**Proof**
Let $y \in \mathbb{R}^d$ be column vector. Then
$$
y^TXy = y^Txx^Ty = (x^Ty)^T(x^Ty) = \|x^Ty\|_2^2 \geq 0
$$
Therefore, $X=xx^T$ is positive semi-definite by definition

**Lemma 2**
Let $x \in \mathbb{R}^d$. Then $(x^Tx)I_n \succeq xx^T$.

**Proof**
Let us denote $X =xx^T$, $r = x^Tx = \|x\|_2^2$ and $X' = X-rI_n$.  
Now let's look on calculation of eigenvalues for matrix $X'$.
We will have to calculate determinant of $X'-\lambda I_n = X-r I_n - \lambda I_n = X-(r + \lambda)I$.  
So we can claim that eigenvalues of $X'$ are just eigenvalues of $X$ minus $r$ (let us denote this fact as $eig(X')=eig(X)-r$).  
Then also recap property of definite matrix: matrix is positive definite if and only if all of its eigenvalues are positive.

Therefore, we need to prove that eigenvalues of $-X' = rI_n - X$ are nonnegative, what means that eigenvalues of $X' = X - rI_n $ are non-positive.  

It is sufficient to show that the maximum eigenvalue $eig_{max}(X')$ is non-positive. We have shown that $eig(X')=eig(X)-r$.  
As $r \geq 0$, $eig_{max}(X') = eig_{max}(X') - r$.  
Note that $Sum(eig(X))=Tr(X)= r$, and, by *Lemma 1*, $X \succeq 0$, so all eigenvalue of $X$ are nonnegative.  
Therefore, the maximum possible $eig_{max}(X') \leq r$ and $eig_{max}(X') = eig_{max}(X') - r \leq 0$.  

We has proven that $eig_{max}(X') \leq 0$, so $eig(X') \leq 0$ and $eig(-X') \geq 0$. Therefore, $-X' = (x^Tx)I_n - xx^T \succeq 0$

Let us define the function $f$ as
$$ f = \frac{1}{n} \sum\limits_{i=1}^n \ell (g(w, x_i), y_i) + \frac{\lambda}{2} \| w \|^2_2$$

So the initial problem is minimizing $f$. Let us also use the following notations:
$$
e_i = e^{-y_iw^Tx_i} \\
h_i(w) = \ell (g(w, x_i), y_i) = ln(1+e^{-y_iw^Tx_i}) = ln(1+e_i) \\
r(w) = \frac{\lambda}{2} \| w \|^2_2 = \frac{\lambda}{2} w^Tw
$$

To compute $\nabla_w f$ and $\nabla_w^2 f$, we first need to find $\nabla h_i$, $\nabla^2 h_i$, $\nabla r$ and $\nabla^2 r$.
Note that $e_i' = -y_ie_ix_i$.

Let us start with $h(w)$:
$$
\nabla h_i = \frac{-y_ie_ix_i}{1+e_i} \\
\nabla^2 h_i = \frac{1}{(1+e_i)^2} (-y_i(-y_ie_ix_i)x_i(1+e_i)-(-y_ie_ix_i)(-y_ie_ix_i)) = \\
= \frac{y_i^2e_ix_ix_i^T}{(1+e_i)^2} (1+e_i-e_i) =  \{ y_i^2 = 1 \text{ as } y_i = \pm 1 \} = \frac{e_ix_ix_i^T}{(1+e_i)^2}
$$

For $r(w)$,
$$
\nabla r = \lambda w\\
\nabla^2 r = \lambda I_n
$$

Therefore,
$$
\nabla f = \frac{1}{n} \sum\limits_{i=1}^n \nabla h_i(w) +  \nabla r(w) = \frac{1}{n} \sum\limits_{i=1}^n \frac{-y_ie_ix_i}{1+e_i} +  \lambda w\\

\nabla^2 f = \frac{1}{n} \sum\limits_{i=1}^n \nabla^2 h_i(w) + \nabla^2 r(w) =\frac{1}{n} \sum\limits_{i=1}^n \frac{e_ix_ix_i^T}{(1+e_i)^2} + \lambda I_n
$$

From Theorem 2.1.6 and Theorem 2.1.11 from Nesterov's book (check references) we know that $f$ is $\mu$-strongly convex and has $L$-Lipschitz gradient iff
$$
L  I_n \succeq \nabla^2 f \succeq \mu I_n
$$

Note that $e_i > 0$, so $\frac{1}{n} \sum\limits_{i=1}^n \frac{e_ix_ix_i^T}{(1+e_i)^2} + \lambda I_n \succeq \lambda I_n$  (using *Lemma 1*).
Therefore $\nabla^2 f \succeq \lambda I_n$ and $f$ is $\mu$-strongly convex with $\mu = \lambda$.

Let us prove that $f$ has $L$-Lipschitz gradient with $L = \lambda + \frac{1}{4n} \sum_{i=1}^n x_i^T x_i$. To do this we need to show that
$$
(\lambda + \frac{1}{4n} \sum_{i=1}^n x_i^T x_i)I_n \succeq \frac{1}{n} \sum\limits_{i=1}^n \frac{e_ix_ix_i^T}{(1+e_i)^2} + \lambda I_n \\
\Longleftrightarrow \\
(\frac{1}{4n} \sum_{i=1}^n x_i^T x_i)I_n \succeq \frac{1}{n} \sum\limits_{i=1}^n \frac{e_ix_ix_i^T}{(1+e_i)^2}
$$

Let us focus on some fixed $x_i$ and prove
$$(\frac{1}{4n} x_i^T x_i)I_n \succeq \frac{1}{n} \frac{e_ix_ix_i^T}{(1+e_i)^2}$$

Using the fact that $x_i^T x_i I_n \succeq  x_ix_i^T$ from *Lemma 2*, we can compare only matrix scalars and proceed with following
$$
\frac{1}{4n} \geq \frac{1}{n} \frac{e_i}{(1+e_i)^2} \\
\frac{1}{4} \geq  \frac{e_i}{(1+e_i)^2} \\
(1+e_i)^2 \geq 4e_i \\
(1-e_i)^2 \geq 0 \\
$$

The last statement is true for all $x_i$, so $L I_n \succeq \nabla^2 f$ is also true and $f$ has $L$-Lipschitz gradient with $L = \lambda + \frac{1}{4n} \sum_{i=1}^n x_i^T x_i$.

**Results**
- $\nabla f = \frac{1}{n} \sum\limits_{i=1}^n \frac{-y_ie_ix_i}{1+e_i} +  \lambda w$
- The problem is $\mu$-strongly convex with $\mu = \lambda$
- The problem has $L$-Lipschitz gradient $L = \lambda + \frac{1}{4n} \sum_{i=1}^n \| x_i\|^2_2$


### Code

In [19]:
class BinaryLogisticRegression(Problem):
    @staticmethod
    def calculate_parameters(
        lambda_ratio: float, x_values: np.ndarray
    ) -> tuple[float, float, float]:
        """Calculate required parameters
        for the binary logistic regression problem

        Args:
            lambda_ratio (float): defined as `lambda_term = lambda_ratio * lipschitz`
            x_values (np.ndarray): training part

        Returns:
            tuple[float,float,float]: (lambda, mu, lipschitz)
        """

        products_sum = 0
        for x in x_values:
            products_sum += x @ x

        lipschitz = float(products_sum / (4 * x_values.shape[0] * (1 - lambda_ratio)))
        lambda_term = lambda_ratio * lipschitz
        mu = lambda_term

        return lambda_term, mu, lipschitz

    def _expand_dim(self, x: np.ndarray) -> np.ndarray:
        """Convert (n,) vector to (n,1)"""
        return np.expand_dims(x, axis=1)

    def _get_iterate_data(
        self, indices: Optional[list[int]] = None
    ) -> list[tuple[np.ndarray, float]]:
        if indices is None:
            return self.data
        return [self.data[idx] for idx in indices]

    def _custom_exponent(self, x: np.ndarray, y: float, w: np.ndarray) -> float:
        return np.exp(-y * w.dot(x))

    def __init__(
        self,
        xs: np.ndarray,
        ys: np.ndarray,
        lambda_term: float,
        seed: float = MANUAL_SEED,
    ) -> None:
        np.random.seed(seed)

        self.xs = xs
        self.ys = ys
        self.lambda_term = lambda_term

        self.data = list(zip(self.xs, self.ys))

        self.grad_shape = self.xs.shape[1]
        self.hes_shape = (self.xs.shape[1], self.xs.shape[1])
        self.sample_size = self.ys.shape[0]

        self.identity = np.identity(self.grad_shape)

    def get_uniformly_sampled_indices(self, n: int = 1) -> list[int]:
        return [np.random.randint(0, len(self.data)) for _ in range(n)]

    def get_value(self, w: np.ndarray, indices: Optional[list[int]] = None) -> float:
        sum_result = 0

        for x, y in self._get_iterate_data(indices):
            sum_result += np.log(1 + self._custom_exponent(x, y, w))

        squared_norm: float = w.dot(w)

        return sum_result / self.sample_size + squared_norm * self.lambda_term / 2

    def get_gradient(
        self, w: np.ndarray, indices: Optional[list[int]] = None
    ) -> np.ndarray:
        sum_result = np.zeros(self.grad_shape)
        for x, y in self._get_iterate_data(indices):
            custom_exp = self._custom_exponent(x, y, w)
            sum_result += (-y * custom_exp * x) / (1 + custom_exp)

        return sum_result / self.sample_size + self.lambda_term * w

In [20]:
blr_lambda, nlr_mu, blr_lipschitz = BinaryLogisticRegression.calculate_parameters(
    1e-3, train_values
)
blr_function = BinaryLogisticRegression(train_values, train_targets, blr_lambda)

w = np.ones(train_values.shape[1])

print(f"{blr_function.get_value(w)=}")
print(f"{blr_function.get_gradient(w).shape=}")

blr_function.get_value(w)=176.55849319150303
blr_function.get_gradient(w).shape=(13,)


## Methods

Maximum iterations: 1000

Required precision $\varepsilon  =10^{-5}$

$\frac{\| \nabla f(x^k) \|}{\| \nabla f(x^0) \|}$ is used as default convergence criterion.

In [17]:
np.random.seed(MANUAL_SEED)
START_POINT = np.random.randn(train_values.shape[1])

NUM_ITERATIONS = 1000
EPSILON = 1e-5


def default_criterion(grad_0: np.ndarray, grad_current: np.ndarray) -> float:
    return float(np.linalg.norm(grad_current) / np.linalg.norm(grad_0))

In [13]:
class Method:
    def _predict(self, w: np.ndarray, x: np.ndarray) -> float:
        # As y can be 1 and -1 only, we are interested only in sign
        return np.sign(w.dot(x))

    def _log_accuracy(self) -> float:
        accuracy = 0
        if not self.accuracy_logging:
            return accuracy

        predictions = np.array([self._predict(self.w_curr, x) for x in self.test_values])

        if len(predictions) == 0:
            self.accuracy_logs.append(accuracy)
            return accuracy

        accuracy = (predictions == self.test_targets).sum() / self.test_targets.shape[0]
        self.accuracy_logs.append(accuracy)
        return accuracy

    def __init__(
        self,
        name: str,
        f: Problem,
        w_0: np.ndarray,
        iterations: int,
        epsilon: float = EPSILON,
        verbose: bool = False,
    ) -> None:
        """Method base class

        Args:
            name (str): Method name
            f (Problem)
            x_0 (np.ndarray): starting point
            iterations (int): number of iterations
            epsilon (float):Required precision. Defaults to EPSILON
            verbose (bool): Defaults to False
        """

        self.name = name
        self.f = f
        self.w_0 = w_0.copy()
        self.iterations = iterations
        self.epsilon = epsilon
        self.verbose = verbose

        self.accuracy_logging = False

        self.reset()

    def enable_accuracy_logging(self, test_values: np.ndarray, test_targets: np.ndarray):
        self.accuracy_logging = True
        self.test_values = test_values
        self.test_targets = test_targets

    def disable_accuracy_logging(self):
        self.accuracy_logging = False

    def predict(self, values: np.ndarray) -> np.ndarray:
        return np.array([self._predict(self.w_sol, x) for x in values])

    def get_name(self) -> str:
        return self.name

    def get_solution(self) -> np.ndarray:
        return self.w_sol

    def get_iterations(self) -> int:
        return self.iterations

    def get_pass_iterations(self) -> int:
        return self.pass_iterations

    def get_time_logs(self) -> list[float]:
        return self.time_logs

    def get_accumulated_time_logs(self) -> list[float]:
        return list(accumulate(self.time_logs, operator.add))

    def get_accuracy_logs(self) -> list[float]:
        return self.accuracy_logs

    def get_criterion_logs(self) -> list[float]:
        return self.criterion_logs

    def update(self, iteration: int) -> bool:
        raise NotImplementedError

    def calculate_criterion(self, k: int) -> float:
        grad_current = self.f.get_gradient(self.w_curr)
        return default_criterion(self.grad_0, grad_current)

    def reset(self):
        self.criterion_logs = []
        self.time_logs = []
        self.accuracy_logs = []

        self.pass_iterations = 0

        self.w_prev = self.w_0.copy()
        self.w_curr = self.w_0.copy()
        self.w_sol = self.w_0.copy()
        self.grad_0 = self.f.get_gradient(self.w_0)

    def start(self) -> np.ndarray:
        """Start iterating

        Returns:
            np.ndarray: solution
        """
        self.reset()

        loop = range(self.iterations)
        if not self.verbose:
            loop = tqdm(range(self.iterations), desc=self.name, leave=True, position=0)

        for k in loop:
            start_time = time.time()
            succeeded = self.update(k)
            finish_time = time.time()

            if not succeeded:  # diverges
                break

            self.pass_iterations += 1

            # logging
            elapsed_time = finish_time - start_time
            criterion_value = self.calculate_criterion(k)
            self.time_logs.append(elapsed_time)
            self.criterion_logs.append(criterion_value)
            accuracy = self._log_accuracy()

            if not self.verbose:
                info_dict = {"time": elapsed_time, "criterion": criterion_value}
                if self.accuracy_logging:
                    info_dict.update({"accuracy": accuracy})
                loop.set_postfix(info_dict)  # type: ignore

            if np.isnan(criterion_value):  # diverges
                break
            if criterion_value < self.epsilon:
                break

        self.w_sol = self.w_curr.copy()

        return self.w_sol

### Loopless SVRG

In [ ]:
class LSVRG(Method):
    def __init__(
        self,
        name: str,
        f: Problem,
        w_0: np.ndarray,
        iterations: int,
        step_size: Callable[[int], float],
        probability: float,
        epsilon: float,
        verbose: bool,
    ) -> None:
        super().__init__(name, f, w_0, iterations, epsilon, verbose)
        self.step_size = step_size
        self.probability = probability

    def reset(self):
        super().reset()
        self.v_curr = self.w_0.copy()  # w in paper
        self.v_grad: np.ndarray = self.f.get_gradient(
            self.v_curr
        )  # full gradient in w_curr

    def get_g(self) -> np.ndarray:
        random_idx = self.f.get_uniformly_sampled_indices()
        return (
            self.f.get_gradient(self.w_curr, random_idx)
            - self.f.get_gradient(self.w_curr, random_idx)
            + self.v_grad
        )

    def update_v(self):
        if np.random.random() < self.probability:
            self.v_curr = self.w_curr.copy()
            self.v_grad = self.f.get_gradient(self.v_curr)

    def update(self, iteration: int) -> bool:
        w_next = self.w_curr - self.step_size(iteration) * self.get_g()
        self.update_v()

        self.w_curr = w_next.copy()
        return True

## Utilities

In [16]:
PREDEFINED_COLORS = [
    "#ff0000",
    "#ffd700",
    "#00ff00",
    "#04eafc",
    "#071efb",
    "#9603fd",
]


def draw_method_plots(
    methods: list[Method],
    title: str,
    ylim=None,
    figsize=(16, 12),
    use_rainbow: bool = False,
):
    if use_rainbow:
        num_colors = len(methods)
        cm = plt.get_cmap("gist_rainbow")
        colors = [cm(1.0 * i / num_colors) for i in range(num_colors)]
    else:
        colors = PREDEFINED_COLORS

    style_cycler = cycler(linestyle=["-", "--", ":", "-."]) * cycler(color=colors)

    _, axs = plt.subplots(3, 1, figsize=figsize, gridspec_kw={"height_ratios": [5, 5, 3]})
    (ax1, ax2, ax3) = axs
    ax1.set_title(f"{title} | Criterion / Iteration")
    ax2.set_title(f"{title} | Criterion / Time")
    ax3.set_title(f"{title} | Elapsed time")

    ax1.set(xlabel="Iteration", ylabel="Criterion value, log scale")
    ax2.set(xlabel="Running time, seconds", ylabel="Criterion value, log scale")
    ax3.set(xlabel="Iteration", ylabel="Elapsed time")

    ax1.set_yscale("log")
    ax2.set_yscale("log")

    for ax in axs.flat:
        ax.grid()
        ax.set_prop_cycle(style_cycler)

    for method in methods:
        label = method.get_name()

        criterion = method.get_criterion_logs()
        iterations = range(method.get_pass_iterations())
        accumulated_time = method.get_accumulated_time_logs()
        elapsed_time = method.get_time_logs()

        ax1.plot(iterations, criterion, label=label)
        ax1.scatter(iterations[-1], criterion[-1], s=15)

        ax2.plot(accumulated_time, criterion, label=label)
        ax2.scatter(accumulated_time[-1], criterion[-1], s=15)

        ax3.plot(iterations, elapsed_time, label=label)

    if len(methods) > 1:
        for ax in axs.flat:
            ax.legend(loc="upper right", bbox_to_anchor=(1.3, 1))

    for ax in axs.flat:
        if ylim is not None:
            ax.set_ylim(top=ylim)

    plt.tight_layout()

## Comparisons